In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, cauchy, gamma, beta
from scipy.special import gamma as gamma_function
import ipywidgets as widgets
from IPython.display import display, HTML

# Set the style
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['text.color'] = 'black'
plt.rcParams['axes.labelcolor'] = 'black'
plt.rcParams['xtick.color'] = 'black'
plt.rcParams['ytick.color'] = 'black'
plt.rcParams['font.family'] = 'monospace' 
plt.rcParams['font.size'] = 8  # font size
plt.rcParams['font.weight'] = 'normal'  # 'bold', 'light', 'normal'

def format_number(prob):
    if prob >= 0.001:
        return "{:.4f}".format(prob)
    else:
        return "{:.1e}".format(prob)
    
# function to plot PDF for continuous distributions
def plot_pdf_2d(x_values, pdf_values, mean, variance, title, details, x_selected=None, pdf_selected=None):
    plt.figure(figsize=(10, 4), dpi=400)
    
    # Detect if there's an infinite value and adjust accordingly
    if np.isinf(max(pdf_values)):
        max_y = np.max(pdf_values[np.isfinite(pdf_values)]) * 1.1
        pdf_values = np.clip(pdf_values, 0, max_y)
        if pdf_selected is not None and np.isinf(pdf_selected):
            pdf_selected = max_y
    
    if x_selected is not None and pdf_selected is not None:
        plt.scatter(x_selected, pdf_selected, color='red', s=20, zorder=5)
        plt.fill_between(x_values, pdf_values, where=(x_values < x_selected), color=(0.92, 0.92, 0.92))
    
    if not np.isnan(variance):
        plt.axvline(mean, color='grey', linestyle='--', label=f'Mean (μ) = {mean:.2f}', linewidth=1)
    if not np.isnan(variance):
        plt.axvline(mean + np.sqrt(variance), color='orange', linestyle='--', label=f'1σ range', linewidth=1)
        plt.axvline(mean - np.sqrt(variance), color='orange', linestyle='--', linewidth=1)
    
    plt.plot(x_values, pdf_values, label='PDF Curve')
    
    # Modify legend order and add P(X<x) and P(X>x) if x_selected is provided
    handles, labels = plt.gca().get_legend_handles_labels()
    if 'PDF Curve' in labels:
        index_pdf_curve = labels.index('PDF Curve')
        # Reorder to move 'PDF Curve' to the front
        order = [index_pdf_curve] + list(range(index_pdf_curve)) + list(range(index_pdf_curve + 1, len(handles)))
        handles = [handles[i] for i in order]
        labels = [labels[i] for i in order]
        
    if x_selected is not None:
        # Computing P(X<x) and P(X>x)
        dx = x_values[1] - x_values[0]  # assuming uniform spacing
        p_less_than_x = np.sum(pdf_values[x_values < x_selected]) * dx
        p_greater_than_x = np.sum(pdf_values[x_values > x_selected]) * dx
        
        # Adding entries to legend
        line = plt.Line2D([0], [0], linestyle='none')
        handles += [plt.Line2D([0], [0], marker='o', color='red', linestyle='None', markersize=5), line]
        labels += [f'P(X<{x_selected:.2f})={format_number(p_less_than_x)}', 
                   f'P(X>{x_selected:.2f})={format_number(p_greater_than_x)}']
    
    # Reordering the legend to make P(X=x) appear last
    order = list(range(len(handles) - 3)) + [len(handles) - 3, len(handles) - 2, len(handles) - 1]
    handles = [handles[i] for i in order]
    labels = [labels[i] for i in order]
    
    plt.xlabel('X=x')
    plt.ylabel('f(X=x)')
    plt.ylim([0, max(pdf_values)*1.1])
    plt.legend(handles, labels)
    plt.show()
    
    for detail in details:
        print("    " + detail)

def plot_gaussian(x, mu, sigma):
    x_values = np.linspace(mu - 4*sigma, mu + 4*sigma, 1000)
    pdf_values = norm.pdf(x_values, mu, sigma)
    pdf_selected = norm.pdf(x, mu, sigma)
    
    title = 'Gaussian PDF'
    details = [
        f"Equation: f(x) = 1 / (2π)^0.5 * σ * e^(-(x-μ)^2 / (2σ^2))",
        f"Mean (μ) = {mu:.2f}",
        f"Variance (σ^2) = {sigma**2:.2f}",
        # f"P(X={x:.2f}) = 1 / (2π)^0.5 * {sigma:.2f} * e^(-{x-mu:.2f}^2 / (2*{sigma:.2f}^2)) = {format_number(pdf_selected)}"
    ]
    plot_pdf_2d(x_values, pdf_values, mu, sigma**2, title, details, x, pdf_selected)
    
def plot_cauchy(x, m, d):
    x_values = np.linspace(m - 4*d, m + 4*d, 1000)
    pdf_values = cauchy.pdf(x_values, m, d)
    pdf_selected = cauchy.pdf(x, m, d)
    
    title = 'Cauchy PDF'
    details = [
        f"Equation: f(x) = 1 / (π*d) * 1 / (1 + (x-m)^2 / d^2)",
        f"Mean (μ) = undefined",
        f"Variance (σ^2) = undefined",
        # f"P(X={x:.2f}) = 1 / (π*{d:.2f}) * 1 / (1 + {x-m:.2f}^2 / {d:.2f}^2) = {format_number(pdf_selected)}"
    ]
    plot_pdf_2d(x_values, pdf_values, np.nan, np.nan, title, details, x, pdf_selected)

def plot_gamma(x, alpha, theta):
    # Use dense logarithmic spacing for x_values near 0
    x_values = np.linspace(0, gamma.ppf(0.99, alpha, scale=theta), 10000)
    
    pdf_values = gamma.pdf(x_values, alpha, scale=theta)
    pdf_selected = gamma.pdf(x, alpha, scale=theta)
    
    mean = alpha * theta
    variance = alpha * theta**2
    title = 'Gamma PDF'
    details = [
        f"Equation: f(x) = (x^(α-1) * e^(-x/θ)) / (Γ(α) * θ^α)",
        f"Mean (μ) = α * θ = {alpha:.2f} * {theta:.2f} = {mean:.4f}",
        f"Variance (σ^2) = α * θ^2 = {alpha:.2f} * {theta**2:.2f} = {variance:.4f}",
        # f"P(X={x:.2f}) = ({x:.2f}^({alpha-1:.2f}) * e^(-{x:.2f}/{theta:.2f})) / (Γ({alpha:.2f}) * {theta:.2f}^{alpha:.2f}) = {format_number(pdf_selected)}"
    ]
    plot_pdf_2d(x_values, pdf_values, mean, variance, title, details, x, pdf_selected)

def plot_exponential(x, theta):
    alpha_exp = 1  # for exponential
    x_values = np.linspace(0, 4*theta, 1000)
    pdf_values = gamma.pdf(x_values, alpha_exp, scale=theta)
    pdf_selected = gamma.pdf(x, alpha_exp, scale=theta)
    
    mean = theta
    variance = theta**2
    title = 'Exponential PDF'
    details = [
        f"Notes: exponential is a special case of gamma distribution where α = 1",
        f"Equation: f(x) = (1/θ) * e^(-x/θ)",
        f"Mean (μ) = θ = {mean:.4f}",
        f"Variance (σ^2) = θ^2 = {variance:.4f}",
        # f"P(X={x:.2f}) = (1/{theta:.2f}) * e^-{x:.2f}/{theta:.2f} = {format_number(pdf_selected)}"
    ]
    plot_pdf_2d(x_values, pdf_values, mean, variance, title, details, x, pdf_selected)

def plot_chi_square(x, r):
    alpha_chi = r / 2  # for chi-square
    theta_chi = 2  # for chi-square
    x_values = np.linspace(0, gamma.ppf(0.99, alpha_chi, scale=theta_chi), 1000)
    pdf_values = gamma.pdf(x_values, alpha_chi, scale=theta_chi)
    pdf_selected = gamma.pdf(x, alpha_chi, scale=theta_chi)
    
    mean = r
    variance = 2*r
    title = 'Chi-Square PDF'
    details = [
        f"Notes: chi-square is a special case of gamma distribution where α = r/2 and θ = 2",
        f"Equation: f(x) = (1 / (2^(r/2) * Γ(r/2))) * x^{r/2-1} * e^(-x/2)",
        f"Mean (μ) = r = {mean:.4f}",
        f"Variance (σ^2) = 2r = {variance:.4f}",
        # f"P(X={x:.2f}) = (1 / (2^{r/2:.2f} * Γ({r/2:.2f})) * {x:.2f}^{r/2-1} * e^-{x:.2f}/2 = {format_number(pdf_selected)}"
    ]
    plot_pdf_2d(x_values, pdf_values, mean, variance, title, details, x, pdf_selected)
    
def plot_beta(x, alpha, beta_val):
    x_values = np.linspace(0, 1, 1000)
    pdf_values = beta.pdf(x_values, alpha, beta_val)
    pdf_selected = beta.pdf(x, alpha, beta_val)
    
    mean = alpha / (alpha + beta_val)
    variance = (alpha * beta_val) / ((alpha + beta_val)**2 * (alpha + beta_val + 1))
    title = 'Beta PDF'
    details = [
        f"Equation: f(x) = Γ(α+β) / (Γ(α) + Γ(β)) * x^(α-1) * (1-x)^(β-1)",
        f"Mean (μ) = α / (α + β) = {alpha:.2f} / ({alpha:.2f} + {beta_val:.2f}) = {mean:.4f}",
        f"Variance (σ^2) = α * β / ((α + β)^2 * (α + β + 1)) = {alpha:.2f} * {beta_val:.2f} / ({alpha+beta_val:.2f}^2 * {alpha+beta_val+1:.2f}) = {variance:.2f}",
        f"P(X={x:.2f}) = Γ({alpha:.2f}+{beta_val:.2f}) / (Γ({alpha:.2f}) + Γ({beta_val:.2f})) * {x:.2f}^{alpha-1:.2f} * {1-x:.2f}^{beta_val-1:.2f} = {format_number(pdf_selected)}"
    ]
    plot_pdf_2d(x_values, pdf_values, mean, variance, title, details, x, pdf_selected)

# Widgets
distribution_dropdown = widgets.Dropdown(
    options=["Select a distribution", "Gaussian", "Cauchy", "Gamma", "Exponential", "Chi-Square", "Beta"],
    value="Select a distribution",
    description='Distribution:'
)

output_container = widgets.VBox([])  # Container to hold sliders and plots
slider_layout = widgets.Layout(width='100%')

def display_distribution_widgets(change):
    if change['new'] == "Gaussian":
        mu_slider = widgets.FloatSlider(value=0, min=-10, max=10, step=0.01, description='μ (mean):', continuous_update=False, layout=slider_layout)
        sigma_slider = widgets.FloatSlider(value=1, min=0.01, max=10, step=0.01, description='σ (s.d.):', continuous_update=False, layout=slider_layout)
        x_slider_gaussian = widgets.FloatSlider(value=0, min=mu_slider.value - 4*sigma_slider.value + 0.01, max=mu_slider.value + 4*sigma_slider.value - 0.01, 
                                                step=0.01, description='x:', continuous_update=False, layout=slider_layout)

        def update_x_range(*args):
            if(mu_slider.value - 4*sigma_slider.value < x_slider_gaussian.max):
                x_slider_gaussian.min = mu_slider.value - 4*sigma_slider.value + 0.01
                x_slider_gaussian.max = mu_slider.value + 4*sigma_slider.value - 0.01
            else:
                x_slider_gaussian.max = mu_slider.value + 4*sigma_slider.value - 0.01
                x_slider_gaussian.min = mu_slider.value - 4*sigma_slider.value + 0.01
        mu_slider.observe(update_x_range, 'value')
        sigma_slider.observe(update_x_range, 'value')

        gaussian_box = widgets.HBox([x_slider_gaussian, mu_slider, sigma_slider])
        gaussian_interactive = widgets.interactive(plot_gaussian, x=x_slider_gaussian, mu=mu_slider, sigma=sigma_slider)
        output_container.children = [gaussian_box, gaussian_interactive.children[-1]]
        
    elif change['new'] == "Cauchy":
        m_slider = widgets.FloatSlider(value=0, min=-10, max=10, step=0.01, description='m (location):', continuous_update=False, layout=slider_layout)
        d_slider = widgets.FloatSlider(value=1, min=0.01, max=10, step=0.01, description='d (scale):', continuous_update=False, layout=slider_layout)
        x_slider_cauchy = widgets.FloatSlider(value=0, min=m_slider.value - 4*d_slider.value + 0.01, max=m_slider.value + 4*d_slider.value - 0.01, 
                                              step=0.01, description='x:', continuous_update=False, layout=slider_layout)

        def update_x_range(*args):
            if(m_slider.value - 4*d_slider.value < x_slider_cauchy.max):
                x_slider_cauchy.min = m_slider.value - 4*d_slider.value + 0.01
                x_slider_cauchy.max = m_slider.value + 4*d_slider.value - 0.01
            else:
                x_slider_cauchy.max = m_slider.value + 4*d_slider.value - 0.01
                x_slider_cauchy.min = m_slider.value - 4*d_slider.value + 0.01
        m_slider.observe(update_x_range, 'value')
        d_slider.observe(update_x_range, 'value')

        cauchy_box = widgets.HBox([x_slider_cauchy, m_slider, d_slider])
        cauchy_interactive = widgets.interactive(plot_cauchy, x=x_slider_cauchy, m=m_slider, d=d_slider)
        output_container.children = [cauchy_box, cauchy_interactive.children[-1]]

    elif change['new'] == "Gamma":
        alpha_slider = widgets.FloatSlider(value=1, min=0.01, max=10, step=0.01, description='α (shape):', continuous_update=False, layout=slider_layout)
        theta_slider = widgets.FloatSlider(value=1, min=0.01, max=10, step=0.01, description='θ (scale):', continuous_update=False, layout=slider_layout)
        x_slider_gamma = widgets.FloatSlider(value=2, min=0.01, max=gamma.ppf(0.99, alpha_slider.value, scale=theta_slider.value) - 0.01, 
                                             step=0.01, description='x:', continuous_update=False, layout=slider_layout)

        def update_x_range(*args):
            if gamma.ppf(0.99, alpha_slider.value, scale=theta_slider.value) - 0.01 < x_slider_gamma.min:
                x_slider_gamma.min = 0.00
                x_slider_gamma.max = 0.002
            else:
                x_slider_gamma.max = gamma.ppf(0.99, alpha_slider.value, scale=theta_slider.value) - 0.01
        alpha_slider.observe(update_x_range, 'value')
        theta_slider.observe(update_x_range, 'value')

        gamma_box = widgets.HBox([x_slider_gamma, alpha_slider, theta_slider])
        gamma_interactive = widgets.interactive(plot_gamma, x=x_slider_gamma, alpha=alpha_slider, theta=theta_slider)
        output_container.children = [gamma_box, gamma_interactive.children[-1]]
        
    elif change['new'] == "Exponential":
        theta_slider_exp = widgets.FloatSlider(value=1, min=0.01, max=10, step=0.01, description='θ (scale):', continuous_update=False, layout=slider_layout)
        x_slider_exp = widgets.FloatSlider(value=1, min=0.01, max=4*theta_slider_exp.value - 0.01, step=0.01, description='x:', continuous_update=False, layout=slider_layout)
        
        def update_x_range(*args):
            x_slider_exp.max = 4*theta_slider_exp.value - 0.01
        theta_slider_exp.observe(update_x_range, 'value')

        exp_box = widgets.HBox([x_slider_exp, theta_slider_exp])
        exp_interactive = widgets.interactive(plot_exponential, x=x_slider_exp, theta=theta_slider_exp)
        output_container.children = [exp_box, exp_interactive.children[-1]]

    elif change['new'] == "Chi-Square":
        r_slider_chi = widgets.FloatSlider(value=2, min=1, max=20, step=1, description='r (d.o.f):', continuous_update=False, layout=slider_layout)
        x_slider_chi = widgets.FloatSlider(value=2, min=0.01, max=gamma.ppf(0.99, r_slider_chi.value/2, scale=2) - 0.01, step=0.01, description='x:', continuous_update=False, layout=slider_layout)
        
        def update_x_range(*args):
            x_slider_chi.max = gamma.ppf(0.99, r_slider_chi.value/2, scale=2) - 0.01
        r_slider_chi.observe(update_x_range, 'value')

        chi_box = widgets.HBox([x_slider_chi, r_slider_chi])
        chi_interactive = widgets.interactive(plot_chi_square, x=x_slider_chi, r=r_slider_chi)
        output_container.children = [chi_box, chi_interactive.children[-1]]
        
    elif change['new'] == "Beta":
        alpha_slider_beta = widgets.FloatSlider(value=2, min=0.01, max=10, step=0.01, description='α (shape):', continuous_update=False, layout=slider_layout)
        beta_val_slider = widgets.FloatSlider(value=5, min=0.01, max=10, step=0.01, description='β (shape):', continuous_update=False, layout=slider_layout)
        x_slider_beta = widgets.FloatSlider(value=0.5, min=0.01, max=0.99, 
                                            step=0.01, description='x:', continuous_update=False, layout=slider_layout)

        beta_box = widgets.HBox([x_slider_beta, alpha_slider_beta, beta_val_slider])
        beta_interactive = widgets.interactive(plot_beta, x=x_slider_beta, alpha=alpha_slider_beta, beta_val=beta_val_slider)
        output_container.children = [beta_box, beta_interactive.children[-1]]
        
# Observer
distribution_dropdown.observe(display_distribution_widgets, names='value')

# Display initial dropdown and the output container
display(distribution_dropdown, output_container)